In [50]:
# If notebook is run on google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb

from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt


#### Import Higgs

In [12]:
df_higgs = pd.read_csv("/content/drive/MyDrive/00 UvA MASTER DATA SCIENCE /05. Applied Machine Learning /higgs_train.csv")
higgs_test = pd.read_csv("/content/drive/MyDrive/00 UvA MASTER DATA SCIENCE /05. Applied Machine Learning /benchmark-tabular-ml/higgs_test.csv")  # adjust path as necessary
higgs_test_submission = pd.read_csv("/content/drive/MyDrive/00 UvA MASTER DATA SCIENCE /05. Applied Machine Learning /benchmark-tabular-ml/higgs_test_submission.csv")

# drop EventId column -> just some Id we dont need
df_higgs = df_higgs.drop(['EventId'], axis=1)
higgs_test_data = higgs_test.drop(['EventId'], axis=1)

# change target variable
df_higgs['Label'] = df_higgs['Label'].replace({'b': 0, 's': 1})

# target variable
higgs_target_variable = 'Label'

#### Import Heloc

In [17]:
path = "/content/drive/MyDrive/00 UvA MASTER DATA SCIENCE /05. Applied Machine Learning /benchmark-tabular-ml/"

df_heloc = pd.read_csv(path+"heloc_train.csv")
synthetic_data = pd.read_csv(path+'synthetic_data_heloc_2.csv')
heloc_test = pd.read_csv(path+"heloc_test.csv")  # adjust path as necessary
heloc_test_submission = pd.read_csv(path+"heloc_test_submission.csv")

# change target variable
df_heloc['RiskPerformance'] = df_heloc['RiskPerformance'].replace(('Bad', 'Good'), (0, 1))

# target variale
heloc_target_variable = 'RiskPerformance'


##### Heloc Synthetic EDA

In [ ]:
from sklearn.utils import shuffle  # For shuffling the dataset

# Concatenate the datasets
combined_data = pd.concat([df_heloc, synthetic_data], ignore_index=True)

# Optional: Shuffle the combined dataset
combined_data = shuffle(combined_data, random_state=42)  # '42' is just an example seed for reproducibility

# Reset index after shuffling (optional but recommended)
combined_data.reset_index(drop=True, inplace=True)

df_heloc['RiskPerformance'] = df_heloc['RiskPerformance'].replace(('Bad', 'Good'), (0, 1))
combined_data['RiskPerformance'] = combined_data['RiskPerformance'].replace(('Bad', 'Good'), (0, 1))
synthetic_data['RiskPerformance'] = df_heloc['RiskPerformance'].replace(('Bad', 'Good'), (0, 1))


#### Import CoverType

In [42]:
path = "/content/drive/MyDrive/00 UvA MASTER DATA SCIENCE /05. Applied Machine Learning /benchmark-tabular-ml/"

df_cov = pd.read_csv(path+"covtype_train.csv")
cov_test = pd.read_csv(path+"covtype_test.csv")  # adjust path as necessary
cov_test_submission = pd.read_csv(path+"covtype_test_submission.csv")
cov_target_variable = 'Cover_Type'

#### NN + Smote + Scaling (CoverType)

In [46]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batchnorm2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

def train_and_predict_multiclass_neural_network(data, target_col, test_data, test_submission, document_name,num_epochs=10, batch_size=64, learning_rate=0.001):
    # Prepare training data
    X_train = data.drop(target_col, axis=1).values
    y_train = data[target_col].values - 1  # Zero-indexing the target
    X_test = test_data.values

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply SMOTE
    smote = SMOTE()
    X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_smote, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_smote, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

    # Initialize the model
    input_size = X_train_smote.shape[1]
    model = NeuralNet(input_size)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Training Loop
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Predictions
    model.eval()
    test_predictions = []
    with torch.no_grad():
        for inputs in DataLoader(X_test_tensor, batch_size=batch_size):
            outputs = model(inputs)
            predicted = torch.argmax(outputs, axis=1)
            test_predictions.extend(predicted.numpy())

    # Correcting for zero-indexing
    test_predictions_corrected = np.array(test_predictions) + 1

        # Add corrected predictions to test_submission DataFrame
    test_submission['Prediction'] = test_predictions_corrected

    # Save the updated DataFrame to CSV
    test_submission.to_csv(document_name+'.csv', index=False)

    return test_submission


In [47]:
document_name = "covtype_NN(sm+sc)"
result_submission = train_and_predict_multiclass_neural_network(df_cov, cov_target_variable, cov_test, cov_test_submission, document_name)

Epoch [1/10], Loss: 0.8035
Epoch [2/10], Loss: 0.5344
Epoch [3/10], Loss: 0.2489
Epoch [4/10], Loss: 0.2331
Epoch [5/10], Loss: 0.4406
Epoch [6/10], Loss: 0.3875
Epoch [7/10], Loss: 0.2461
Epoch [8/10], Loss: 0.3960
Epoch [9/10], Loss: 0.5543
Epoch [10/10], Loss: 0.3304


#### NN No Smote, No Scaling (CovType)

In [48]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batchnorm2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

def train_and_predict_multiclass_neural_network_no_smsc(data, target_col, test_data, test_submission, document_name,num_epochs=10, batch_size=64, learning_rate=0.001):
    # Prepare training data
    X_train = data.drop(target_col, axis=1).values
    y_train = data[target_col].values - 1  # Zero-indexing the target
    X_test = test_data.values

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

    # Initialize the model
    input_size = X_train.shape[1]
    model = NeuralNet(input_size)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Training Loop
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Predictions
    model.eval()
    test_predictions = []
    with torch.no_grad():
        for inputs in DataLoader(X_test_tensor, batch_size=batch_size):
            outputs = model(inputs)
            predicted = torch.argmax(outputs, axis=1)
            test_predictions.extend(predicted.numpy())

    # Correcting for zero-indexing
    test_predictions_corrected = np.array(test_predictions) + 1

        # Add corrected predictions to test_submission DataFrame
    test_submission['Prediction'] = test_predictions_corrected

    # Save the updated DataFrame to CSV
    test_submission.to_csv(document_name+'.csv', index=False)

    return test_submission

In [49]:
document_name = "covtype_NN(unprocessed)"
result_submission = train_and_predict_multiclass_neural_network_no_smsc(df_cov, cov_target_variable, cov_test, cov_test_submission, document_name)

Epoch [1/10], Loss: 0.8674
Epoch [2/10], Loss: 0.7277
Epoch [3/10], Loss: 0.6986
Epoch [4/10], Loss: 0.6831
Epoch [5/10], Loss: 0.8821
Epoch [6/10], Loss: 0.8115
Epoch [7/10], Loss: 0.6387
Epoch [8/10], Loss: 0.7415
Epoch [9/10], Loss: 0.6509
Epoch [10/10], Loss: 0.6115


#### NN + Smote + Scaling (Heloc, Higgs)

In [20]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

def train_and_predict_neural_network(data, target_col, test_data, test_submission, document_name, num_epochs=10, batch_size=64, learning_rate=0.001):
    # Prepare training data
    X_train = data.drop(target_col, axis=1).values
    y_train = data[target_col].values
    X_test = test_data.values

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply SMOTE
    smote = SMOTE()
    X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_smote, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_smote, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

    # Define the model
    input_size = X_train_smote.shape[1]
    model = NeuralNet(input_size)

    # Loss and Optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Training Loop
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(inputs)
            labels = labels.unsqueeze(1)  # Adjust labels' shape if necessary
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Predictions
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test_tensor)
        test_predictions = (test_predictions >= 0.5).float().squeeze()  # Convert to binary predictions

    # Add predictions to test_submission DataFrame
    test_submission['Prediction'] = test_predictions.numpy()

    # Save the updated DataFrame to CSV
    test_submission.to_csv(document_name+'.csv', index=False)

    return test_submission

In [22]:
document_name = "higgs_NN(sm+sc)"

result_submission_higgs = train_and_predict_neural_network(df_higgs, higgs_target_variable, higgs_test_data, higgs_test_submission, document_name)

Epoch [1/10], Loss: 0.1577
Epoch [2/10], Loss: 0.0921
Epoch [3/10], Loss: 0.0162
Epoch [4/10], Loss: 0.0048
Epoch [5/10], Loss: 0.0061
Epoch [6/10], Loss: 0.0004
Epoch [7/10], Loss: 0.0664
Epoch [8/10], Loss: 0.0007
Epoch [9/10], Loss: 0.0041
Epoch [10/10], Loss: 0.0199


In [21]:
document_name = "heloc_NN(sm+sc)"

result_submission_heloc = train_and_predict_neural_network(df_heloc, heloc_target_variable, heloc_test, heloc_test_submission, document_name)

Epoch [1/10], Loss: 0.5635
Epoch [2/10], Loss: 0.5938
Epoch [3/10], Loss: 0.6430
Epoch [4/10], Loss: 0.6089
Epoch [5/10], Loss: 0.4911
Epoch [6/10], Loss: 0.5258
Epoch [7/10], Loss: 0.3464
Epoch [8/10], Loss: 0.5952
Epoch [9/10], Loss: 0.5300
Epoch [10/10], Loss: 0.5595


#### Neural Net No Smote + No Scaling (Heloc & Higgs)

In [25]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

def train_and_predict_neural_network_no_smsc(data, target_col, test_data, test_submission, document_name, num_epochs=10, batch_size=64, learning_rate=0.001):
    # Prepare training data
    X_train = data.drop(target_col, axis=1).values
    y_train = data[target_col].values
    X_test = test_data.values

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

    # Define the model
    input_size = X_train.shape[1]
    model = NeuralNet(input_size)

    # Loss and Optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Training Loop
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(inputs)
            labels = labels.unsqueeze(1)  # Adjust labels' shape if necessary
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Predictions
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test_tensor)
        test_predictions = (test_predictions >= 0.5).float().squeeze()  # Convert to binary predictions

    # Add predictions to test_submission DataFrame
    test_submission['Prediction'] = test_predictions.numpy()

    # Save the updated DataFrame to CSV
    test_submission.to_csv(document_name+'.csv', index=False)

    return test_submission

In [26]:
document_name = "heloc_NN(unprocessed)"

result_submission_heloc_unprocessed = train_and_predict_neural_network_no_smsc(df_heloc, heloc_target_variable, heloc_test, heloc_test_submission, document_name)

Epoch [1/10], Loss: 0.3543
Epoch [2/10], Loss: 0.2676
Epoch [3/10], Loss: 0.8077
Epoch [4/10], Loss: 0.4615
Epoch [5/10], Loss: 0.7588
Epoch [6/10], Loss: 0.5597
Epoch [7/10], Loss: 0.6640
Epoch [8/10], Loss: 0.8600
Epoch [9/10], Loss: 0.5038
Epoch [10/10], Loss: 0.5895


In [27]:
document_name = "higgs_NN(unprocessed)"

result_submission_higgs = train_and_predict_neural_network_no_smsc(df_higgs, higgs_target_variable, higgs_test_data, higgs_test_submission, document_name)

Epoch [1/10], Loss: 0.2709
Epoch [2/10], Loss: 0.1041
Epoch [3/10], Loss: 0.0186
Epoch [4/10], Loss: 0.3512
Epoch [5/10], Loss: 0.1263
Epoch [6/10], Loss: 0.0201
Epoch [7/10], Loss: 0.1708
Epoch [8/10], Loss: 0.0596
Epoch [9/10], Loss: 0.0286
Epoch [10/10], Loss: 0.2828
